# Sku Generator Beta Testing Playground

Read Apple Numbers file with multiple sheets into Pandas Data frames

In [1]:
import pandas as pd
from numbers_parser import Document

## Data Preparation and Validation

In [4]:
def load_catalog_data(file_path="",debug=False):
    """
    Load Apple Numbers file with multiple sheets into Pandas DataFrames.
    
    Parameters:
    - file_path: path to the Numbers file
    
    Returns:
    - tables: dictionary of DataFrames, one for each sheet/table in the Numbers file
    """
    # Read the Numbers file
    doc = Document(file_path)
    sheets = doc.sheets
    tables = {}

    # Extract each sheet as a separate dataframe
    for sheet in sheets:
        for table in sheet.tables:
            table_name = table.name.replace(" ", "_")
            data = table.rows(values_only=True)
            tables[table_name] = pd.DataFrame(data[1:], columns=data[0])

    # Access individual dataframes
    if debug:
        for name, df in tables.items():
            print(f"\n{name}:")
            print(df.head())

    # For Product Catalog sheet replace spaces with underscores in column names
    tables['Product_Catalog'].columns = tables['Product_Catalog'].columns.str.replace(' ', '_')
    
    return tables

# Load the data
file_path = "CHARTS/catalog.numbers"
tables = load_catalog_data(file_path,True)

# Show list of table names
print("\nTables loaded:")
for table_name in tables.keys():
    print(f"- {table_name}")



Product_Catalog:
   Index Main Category           Sub Category                       Name  \
0    0.0         Shirt              Tee Shirt         Bandit Banquet Tee   
1    1.0         Shirt              Tee Shirt    Bandit Truck Wreath Tee   
2    2.0         Shirt              Tee Shirt      Bandit Truck Logo Tee   
3    3.0         Shirt              Tee Shirt              Legendary Tee   
4    4.0         Shirt  Long Sleeve Tee Shirt  Legendary Long Sleeve Tee   

                                                Size     Fit  \
0  Child Small,Child Medium,Child Large,Extra Sma...  Unisex   
1  Child Small,Child Medium,Child Large,Extra Sma...  Unisex   
2  Child Small,Child Medium,Child Large,Extra Sma...  Unisex   
3  Child Small,Child Medium,Child Large,Extra Sma...  Unisex   
4  Child Small,Child Medium,Child Large,Extra Sma...  Unisex   

                      Color                                   Design Material  \
0                    Yellow                           Bandi

In [9]:
def validate_catalog_features(data="",debug=False):
    """
    Validate that code sheet names match catalog features and extract the feature list.
    
    Parameters:
    - tables: dictionary of DataFrames from load_catalog_data()
    
    Returns:
    - tuple: (code_sheet_list, catalog_feature_list)
    
    Raises:
    - AssertionError if sheet names don't match catalog features
    """
    # get lists of code sheets and catalog features
    code_sheet_list = list(tables.keys())[1:]  # List of sheet names (skip Product_Catalog)
    catalog_feature_list = list(tables['Product_Catalog'].columns[1:])  # list of features from Product Catalog sheet excluding first column (Index)

    # remove "Name" from catalog_feature_list if present
    if "Name" in catalog_feature_list:
        catalog_feature_list.remove("Name")
    
    # remove "Prefix_Codes" from code_sheet_list if present (it's a reference table, not a feature)
    if "Prefix_Codes" in code_sheet_list:
        code_sheet_list.remove("Prefix_Codes")

    # sanity check: make sure both lists match exactly
    if set(code_sheet_list) == set(catalog_feature_list):
        print("Sheet names and catalog features MATCH exactly!")
    else:
        raise AssertionError("Sheet names and catalog features DO NOT match!")

    # for visual reference, create a DF to show matching sheets and features
    matching_df = pd.DataFrame({
        'Sheet_Names': code_sheet_list,
        'Catalog_Features': catalog_feature_list
    })
    if debug:
        print(matching_df)
    
    return code_sheet_list, catalog_feature_list

# Validate and extract feature lists
code_sheet_list, catalog_feature_list = validate_catalog_features(tables,True)

Sheet names and catalog features MATCH exactly!
     Sheet_Names Catalog_Features
0           Size    Main_Category
1  Main_Category     Sub_Category
2   Sub_Category             Size
3            Fit              Fit
4          Color            Color
5         Design           Design
6       Material         Material
7          Scent            Scent


In [8]:
# Debug: Check what columns are in Product_Catalog vs what sheets exist
code_sheet_list = list(tables.keys())[1:]  # Skip Product_Catalog
catalog_feature_list = list(tables['Product_Catalog'].columns[1:])  # Skip Index
if "Name" in catalog_feature_list:
    catalog_feature_list.remove("Name")

print("Code sheets:", code_sheet_list)
print("\nCatalog features:", catalog_feature_list)
print("\nIn code_sheet_list but not in catalog_feature_list:", set(code_sheet_list) - set(catalog_feature_list))
print("In catalog_feature_list but not in code_sheet_list:", set(catalog_feature_list) - set(code_sheet_list))

Code sheets: ['Size', 'Prefix_Codes', 'Main_Category', 'Sub_Category', 'Fit', 'Color', 'Design', 'Material', 'Scent']

Catalog features: ['Main_Category', 'Sub_Category', 'Size', 'Fit', 'Color', 'Design', 'Material', 'Scent']

In code_sheet_list but not in catalog_feature_list: {'Prefix_Codes'}
In catalog_feature_list but not in code_sheet_list: set()


## Prepare Data

In [13]:
# create a list of batch names from the set of unique Name field values in the product catalog

def prepare_batches(data):
    """
    Get unique product names from the Product Catalog and check for duplicates.
    
    Parameters:
    - data: DataFrame (Product_Catalog)
    
    Returns:
    - list of unique product names (batch names)
    """
    # Get unique names from the Name column
    name_field_list = data['Name'].tolist()
    name_field_set = set(name_field_list)
    
    # Check for duplicates by comparing list length to set length
    has_duplicates = len(name_field_list) != len(name_field_set)
    
    if has_duplicates:
        duplicate_list = []
        print(f"WARNING: Found {len(name_field_list) - len(name_field_set)} duplicate name(s)")
        # Find and print duplicates
        seen = {}
        for idx, name in enumerate(name_field_list):
            if name in seen:
                duplicate_list.append((name, seen[name], idx))
            else:
                seen[name] = idx
        
        # Print the duplicates
        print("\nDuplicate Names Found:")
        for name, first_idx, second_idx in duplicate_list:
            print(f"  '{name}' at indexes {first_idx} and {second_idx}")
    else:
        print(f"All product names are unique, total batches to process: {len(name_field_list)}")
    
    return list(name_field_set)

batch_name_list = prepare_batches(tables['Product_Catalog'])    

All product names are unique, total batches to process: 18


### Permutation logic
Generate dictionaries all possible permutations of a given prodcut model, extracting the feature, prefix, and index

In [10]:
from itertools import product
import re

def generate_permutations(batch_name="",debug=False):
    """
    For a given product model (ie batch_name), generate all possible permutations of feature combinations.
    Returns a list of dictionaries, where each dictionary represents one permutation.
    Each value in the dict is [value_name, prefix, index] (as strings).
    Prefix and index are from the corresponding code sheet.
    For AND cases (semicolon), indexes are concatenated and value names are simplified.
    Features with index '0' or prefix ending in 'NAN' are excluded (not applicable values).

    Parameters:
    - batch_name: String representing a row from Product_Catalog DataFrame
    - debug: Boolean flag to enable debug printing
    
    Returns:
    - List of dictionaries, each representing a permutation
    """

    # Size abbreviation to full name mapping
    size_mapping = {
        'CS': 'Child Small',
        'CM': 'Child Medium',
        'CL': 'Child Large',
        'XS': 'Extra Small',
        'S': 'Small',
        'M': 'Medium',
        'L': 'Large',
        'XL': 'Extra Large',
        '2XL': 'Double Extra Large',
        '3XL': 'Triple Extra Large',
        '4XL': 'Quadruple Extra Large',
        '5XL': 'Quintuple Extra Large',
        'S/M': 'Small to Medium',
        'L/XL': 'Large to Extra Large',
        'NA': 'Not Applicable'
    }
    
    # Dictionary to hold all feature values for this product model
    feature_options = {}
    
    for feature in catalog_feature_list:
        feature_values = batch_name[feature]
        
        # Check if feature_values contains comma or semicolon (is a list)
        if isinstance(feature_values, str):
            if ',' in feature_values:
                # Comma-separated = "OR" - these create separate permutations
                values = [v.strip() for v in feature_values.split(',')]
                
                # If this is Size feature, map abbreviations to full names
                if feature == 'Size':
                    values = [size_mapping.get(v, v) for v in values]
                
                feature_options[feature] = {
                    'values': values,
                    'type': 'OR'
                }
            elif ';' in feature_values:
                # Semicolon-separated = "AND" - these must all appear together
                values = [v.strip() for v in feature_values.split(';')]
                
                # If this is Size feature, map abbreviations to full names
                if feature == 'Size':
                    values = [size_mapping.get(v, v) for v in values]
                
                feature_options[feature] = {
                    'values': values,
                    'type': 'AND'
                }
            else:
                # Single value
                value = feature_values
                
                # If this is Size feature, map abbreviation to full name
                if feature == 'Size':
                    value = size_mapping.get(value.strip(), value)
                
                feature_options[feature] = {
                    'values': [value],
                    'type': 'SINGLE'
                }
        else:
            # Non-string value (e.g., NaN, number) - treat as single value
            feature_options[feature] = {
                'values': [feature_values] if not pd.isna(feature_values) else [''],
                'type': 'SINGLE'
            }
    
    # Helper function to get prefix and index for a feature value
    def get_value_info(feature, value_name):
        """Returns [value_name, prefix, index] for a given feature value."""
        code_sheet = tables[feature]
        matching_rows = code_sheet.loc[code_sheet["Name"] == value_name]
        
        if not matching_rows.empty:
            code_index = matching_rows.index[0]
            prefix = code_sheet.at[code_index, "Prefix"]
            # Convert index to string (no padding)
            index_str = str(code_index)
            return [value_name, prefix, index_str]
        else:
            # If not found, return empty prefix and empty index
            return [value_name, "", ""]
    
    # Helper function to check if a feature should be excluded
    def is_not_applicable(value_info):
        """Returns True if the feature is 'Not Applicable' (index 0 or prefix ends with NAN)"""
        index = value_info[2]
        prefix = value_info[1]
        return index == '0' or prefix.endswith('NAN')
    
    # Helper function to simplify value names for AND cases
    def simplify_and_values(value_names):
        """
        Simplifies multiple value names by finding common parts.
        E.g., ['Bandit Truck Icon', 'Bandit Truck Wordmark'] -> 'Bandit Truck Icon AND Wordmark'
        """
        if len(value_names) == 1:
            return value_names[0]
        
        # Find common prefix among all names
        common_prefix = ""
        min_len = min(len(name) for name in value_names)
        
        for i in range(min_len):
            if all(name[i] == value_names[0][i] for name in value_names):
                common_prefix += value_names[0][i]
            else:
                break
        
        # Clean up common prefix (remove trailing spaces/incomplete words)
        common_prefix = common_prefix.rstrip()
        
        # Extract unique parts from each name
        unique_parts = []
        for name in value_names:
            unique_part = name[len(common_prefix):].strip()
            if unique_part:
                unique_parts.append(unique_part)
        
        # Construct simplified name
        if common_prefix and unique_parts:
            return f"{common_prefix} {' AND '.join(unique_parts)}"
        else:
            return ' AND '.join(value_names)
    
    # Separate features by type
    or_features = {k: v['values'] for k, v in feature_options.items() if v['type'] == 'OR'}
    and_features = {k: v['values'] for k, v in feature_options.items() if v['type'] == 'AND'}
    single_features = {k: v['values'] for k, v in feature_options.items() if v['type'] == 'SINGLE'}
    
    # Build permutation components
    permutation_dict = {}
    
    # Add single features (these don't multiply permutations)
    for feature, values in single_features.items():
        permutation_dict[feature] = values
    
    # Add OR features (these DO multiply permutations)
    for feature, values in or_features.items():
        permutation_dict[feature] = values
    
    # Generate all permutations using cartesian product
    feature_names = list(permutation_dict.keys())
    feature_value_lists = [permutation_dict[f] for f in feature_names]
    
    permutations = []
    for combo in product(*feature_value_lists):
        perm = {}
        
        # For each feature in this combination, get [value, prefix, index]
        for feature, value_name in zip(feature_names, combo):
            value_info = get_value_info(feature, value_name)
            
            # Skip if this is a "Not Applicable" value
            if not is_not_applicable(value_info):
                perm[feature] = value_info
        
        # Add AND features (combine into single [simplified_name, prefix, concatenated_indexes])
        for feature, values in and_features.items():
            # Get info for all values
            value_infos = [get_value_info(feature, val) for val in values]
            
            # Check if any of the AND values are "Not Applicable"
            if any(is_not_applicable(info) for info in value_infos):
                continue  # Skip this entire AND feature
            
            # Simplify the value name
            simplified_name = simplify_and_values([info[0] for info in value_infos])
            
            # Use first prefix (assuming they're all the same)
            prefix = value_infos[0][1] if value_infos else ""
            
            # Concatenate indexes (no padding)
            concatenated_index = "".join([info[2] for info in value_infos])
            
            perm[feature] = [simplified_name, prefix, concatenated_index]
        
        # Add the product name
        perm['Name'] = batch_name['Name']
        
        permutations.append(perm)

    if debug:
        print(f"Product: {batch_name['Name']}")
        print(f"Number of permutations: {len(permutations)}\n")
        print(f"Data imput type {type(batch_name)}")
        print(f"Data output type: {type(permutations)}\n")
    
    return permutations

# Test permutation generation with arbitrary row
test_row = tables['Product_Catalog'].iloc[2]
permutations = generate_permutations(test_row,True)

Product: Bandit Truck Logo Tee
Number of permutations: 12

Data imput type <class 'pandas.core.series.Series'>
Data output type: <class 'list'>



### SKU number generation
Generate unique SKUs based on the feature prefixes and indices. Use a timestamp with millisecond accuracy as a contingincy in the case of duplicate SKUs

In [11]:
# generate a unique sku based on prefixes and indexes
from datetime import datetime
from multiprocessing.util import debug
import base36

def generate_sku(dict, add_timestamp=False, debug=False):
    """
    Generate a SKU from a permutation dictionary.
    
    Parameters:
    - dict: permutation dictionary with feature info
    - add_timestamp: if True, adds a compact timestamp suffix for guaranteed uniqueness

    Returns:
    - Tuple: model_name, sku
    """

    model_name = dict['Name']
    part_list = []
    sku_prefix = ""

    # Create sku prefix for main prefix, and sub prefix and index
    sku_prefix += dict['Main_Category'][1]
    sku_prefix += dict['Sub_Category'][1]
    sku_prefix += dict['Sub_Category'][2]
    part_list.append(sku_prefix)


    # iterate the rest of the dict items - now using only prefix for all features
    for i, key in enumerate(dict.keys()):
        if i >= 2 and key != "Name":  # only the third and proceeding items, exclude "Name"
            # Use only the prefix (no index) for all features
            part = dict[key][1]
            # Skip empty prefixes
            if part:
                part_list.append(part)

    # Base SKU
    sku = "-".join(part_list)
    
    # Add timestamp suffix if requested
    if add_timestamp:
        now = datetime.now()
        unix_ms = int(now.timestamp() * 1000)
        compact_time = base36.dumps(unix_ms)[-6:].upper()
        
        sku = f"{sku}-{compact_time}"
    
    if debug:
        print(f"Sku generated for\n Product Model: {model_name}\n variant: {sku}\n SKU length: {len(sku)}")

    return (model_name, sku)

# test on an arbitrary permutation
model, sku = generate_sku(permutations[0],False,True)

Sku generated for
 Product Model: Bandit Truck Logo Tee
 variant: SRTTEE1-USX-BLK-CTN-CS-BTI
 SKU length: 26


In [12]:
# Debug: Check what's in the Product_Catalog for this product
print("Test row (Product_Catalog data):")
print(f"Name: {test_row['Name']}")
print(f"Size value: '{test_row['Size']}'")
print(f"Size type: {type(test_row['Size'])}")
print("\nSize table:")
print(tables['Size'])

Test row (Product_Catalog data):
Name: Bandit Truck Logo Tee
Size value: 'Child Small,Child Medium,Child Large,Extra Small,Small,Medium,Large,Extra Large,Double Extra Large,Triple Extra Large,Quadruple Extra Large,Quintuple Extra Large'
Size type: <class 'str'>

Size table:
    Code Prefix                   Name           Comments
0    0.0   SNAN         Not Applicable  Non Apparel Items
1    1.0     CS            Child Small               None
2    2.0     CM           Child Medium               None
3    3.0     CL            Child Large               None
4    4.0     XS            Extra Small               None
5    5.0      S                  Small               None
6    6.0      M                 Medium               None
7    7.0      L                  Large               None
8    8.0     XL            Extra Large               None
9    9.0    2XL     Double Extra Large               None
10  10.0    3XL     Triple Extra Large               None
11  11.0    4XL  Quadruple Ex

## SKU number persistance
Generate and write generate SKUs to a csv file with other product details. 
Prevent duplicate skus by checking the existing CSV and regenerate skus with timestamp based unique identifiers in this case.

### Sequential Permutation and Procedural SKU Generation

Generate a dictionary containing all permutations.

Generate a SKU for each permutation and add it to the respective dictionary.

In [14]:

def generate_batch_dict(data=list,debug=False):
    """
    Generate a dictionary capturing all generated SKU permutations for each batch.
    Parameters:
    - data: list of batch names (product models)
    - debug: boolean flag to enable debug printing
    Returns:
    - batch_dict: dictionary where keys are batch names and values are lists of permutation dictionaries
    """
    
    batch_dict = {} #capture updated batch data with SKUs

    for batch_name in data:
        # Get the row from Product_Catalog that matches this batch_name
        batch_row = tables['Product_Catalog'][tables['Product_Catalog']['Name'] == batch_name].iloc[0]
        
        # Generate permutations for this batch (returns list of dictionaries)
        permed_batch = generate_permutations(batch_row)
        batch_dict[batch_name] = permed_batch
        
        # Add SKU to each permutation dictionary
        for perm in permed_batch:
            model, sku = generate_sku(perm)
            perm['SKU'] = sku  # Add SKU key directly to the permutation dictionary
        
        if debug:
            print(f"Batch '{batch_name}': Generated {len(permed_batch)} SKUs")

    return batch_dict

batch_dict = generate_batch_dict(batch_name_list,True)

Batch 'Legendary Distressed Hat': Generated 2 SKUs
Batch 'Bandit Banquet Tee': Generated 12 SKUs
Batch 'Bandit Truck Wreath Tee': Generated 12 SKUs
Batch 'Bandit Truck Jacket': Generated 12 SKUs
Batch 'Bandit Truck Flex Fit Hat': Generated 2 SKUs
Batch 'Legendary Trucker Hat': Generated 2 SKUs
Batch 'Collectible Gold Keychain': Generated 2 SKUs
Batch 'Legendary Long Sleeve Tee': Generated 12 SKUs
Batch 'Legendary Jacket': Generated 12 SKUs
Batch 'Legendary Tee': Generated 48 SKUs
Batch 'Bandit Truck Logo Tee': Generated 12 SKUs
Batch 'Legendary Sweater': Generated 12 SKUs
Batch 'Bandit Truck Wreath Sweater': Generated 12 SKUs
Batch 'Legendary Flex Fit Hat': Generated 2 SKUs
Batch 'Bandit Truck Trucker Hat': Generated 1 SKUs
Batch 'Legendary Polo': Generated 24 SKUs
Batch 'Leather Air Freshener': Generated 6 SKUs
Batch 'Legendary Hoody': Generated 12 SKUs


Parse a given batch (collection of permutations with skus) into a dataframe

In [16]:
def batch_to_dataframe(batch_name='',data_source=dict, batch_name_list=batch_name_list,debug=False):
    """
    Convert a batch's list of permutation dictionaries to a Pandas DataFrame.
    Parameters:
    - batch_name: String name of the batch to convert
    - data_source: Dictionary of batches
    - batch_name_list: List of valid batch names (default: batch_name_list)
    - debug: Boolean flag to enable debug printing
    Returns:
    - batch_df: Pandas DataFrame for the specified batch
    """
    for batch in data_source:
        if batch == batch_name:
            batch_df = pd.DataFrame(data_source[batch])
            batch_df.name = f"{batch_name}_df"
        # check for duplicate SKUs in the dataframe
            duplicate_skus = batch_df['SKU'][batch_df['SKU'].duplicated()]
            if not duplicate_skus.empty:
                print(f"WARNING: Found {len(duplicate_skus)} duplicate SKU(s) in batch '{batch_name}':")
                for sku in duplicate_skus.unique():
                    duplicate_rows = batch_df[batch_df['SKU'] == sku]
                    row_indices = duplicate_rows.index.tolist()
                    print(f"  SKU: {sku} at rows {row_indices}")
            else:
                print(f"No duplicate SKUs found in batch '{batch_name}'. Total SKUs: {len(batch_df)}\n")

            if debug:
                print(f"Converted batch '{batch_name}' to DataFrame\n name: {batch_name}_df\n Shape: {batch_df.shape}")
    return batch_df


In [ ]:
batch_name = batch_name_list[0]
batch_to_dataframe(batch_name,batch_dict,debug=True) 

No duplicate SKUs found in batch 'Collectible Gold Keychain'. Total SKUs: 2

Converted batch 'Collectible Gold Keychain' to DataFrame
 name: Collectible Gold Keychain_df
 Shape: (2, 8)


,Main_Category,Sub_Category,Size,Color,Material,Design,Name,SKU
0,"[Accessories, ACC, 4]","[Keychain, KYC, 11]","[, , ]","[Gold, GLD, 11]","[Metal, MTL, 5]","[Legendary Icon, LGI, 2]",Collectible Gold Keychain,ACCKYC11--GLD-MTL-LGI
1,"[Accessories, ACC, 4]","[Keychain, KYC, 11]","[, , ]","[Gold, GLD, 11]","[Metal, MTL, 5]","[Bandit Truck Icon, BTI, 4]",Collectible Gold Keychain,ACCKYC11--GLD-MTL-BTI


## Output dataframe to a CSV

Initialize an output

In [17]:
import os
from datetime import datetime

def initialize_output_csv(output_dir='', batch_name='', overwrite=False, debug=False):
    """
    Initialize a CSV file for SKU batch output using a predefined template structure.
    
    Creates a timestamped CSV file with columns matching the sku_batch_template.csv format.
    The filename includes the batch name and date for organization and tracking.
    
    Parameters
    ----------
    output_dir : str
        Directory path where the CSV should be created (e.g., 'Results/').
        Directory will be created if it doesn't exist.
    batch_name : str
        Name of the product model/batch being processed. Used in the filename
        for identification (e.g., 'Basic Tee' -> 'Basic Tee_skus_20251121.csv').
    overwrite : bool, optional
        Controls behavior when file already exists:
        - False (default): Skips file creation and returns None
        - True: Replaces existing file with new empty template
    debug : bool, optional
        If True, prints detailed diagnostic information including file paths,
        directory status, and overwrite mode. Default is False.
    
    Returns
    -------
    str or None
        Absolute path to the created CSV file if successful, or None if file
        already exists and overwrite=False.
    
    Notes
    -----
    - File naming format: '{batch_name}_skus_{YYYYMMDD}.csv'
    - Date-based naming allows multiple generations per batch on different days
    - Template loaded from 'CHARTS/sku_batch_template.csv'
    - All SKU columns from template are preserved in the output file
    
    Examples
    --------
    Create new batch file:
    >>> path = initialize_output_csv('Results/', 'Basic Tee', overwrite=False)
    >>> print(path)
    'Results/Basic Tee_skus_20251121.csv'
    
    Overwrite existing batch file:
    >>> path = initialize_output_csv('Results/', 'Basic Tee', overwrite=True)
    Overwriting existing CSV at: Results/Basic Tee_skus_20251121.csv
    
    Debug mode for troubleshooting:
    >>> path = initialize_output_csv('Results/', 'Basic Tee', debug=True)
    Debug: Attempting to create file at: Results/Basic Tee_skus_20251121.csv
    Debug: Directory exists: True
    Debug: File exists: False
    Debug: Overwrite mode: False
    """

    # timestamp the file creation in the filepath
    timestamp = datetime.now().strftime('%Y%m%d')

    file_path = output_dir + batch_name + '_skus_' + timestamp + '.csv'
    
    if debug:
        print(f"Debug: Attempting to create file at: {file_path}")
        print(f"Debug: Directory exists: {os.path.exists(output_dir)}")
        print(f"Debug: File exists: {os.path.exists(file_path)}")
        print(f"Debug: Overwrite mode: {overwrite}")

    # Check if FILE already exists (not directory)
    if os.path.exists(file_path):
        if not overwrite:
            print(f"CSV already exists at: {file_path}")
            print(f"Set overwrite=True to replace existing file")
            return None
        else:
            print(f"Overwriting existing CSV at: {file_path}")
    
    # Load the template CSV from CHARTS directory
    template_path = 'CHARTS/sku_batch_template.csv'
    template_df = pd.read_csv(template_path)
    template_columns = template_df.columns.tolist()
    
    # Create an empty DataFrame with these columns
    output_df = pd.DataFrame(columns=template_columns)
    
    # Ensure the directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Save to CSV
    output_df.to_csv(file_path, index=False)
    
    print(f"Created new CSV template with columns: {template_columns}")
    print(f"Saved to: {file_path}")
    
    return file_path

# Test the function
output_dir = 'Results/'
file_path = initialize_output_csv(output_dir, model, overwrite=True, debug=True)
print(f"\nReturned: {file_path}")


Debug: Attempting to create file at: Results/Bandit Truck Logo Tee_skus_20251201.csv
Debug: Directory exists: True
Debug: File exists: True
Debug: Overwrite mode: True
Overwriting existing CSV at: Results/Bandit Truck Logo Tee_skus_20251201.csv
Created new CSV template with columns: ['Index', 'SKU', 'Main_Category', 'Sub_Category', 'Name', 'Size', 'Fit', 'Color', 'Design', 'Material', 'Scent']
Saved to: Results/Bandit Truck Logo Tee_skus_20251201.csv

Returned: Results/Bandit Truck Logo Tee_skus_20251201.csv


Populate the output with a given dataframe

In [18]:
def populate_output_csv(file_path='', df=pd.DataFrame(), debug=False):
    """
    Populate the output CSV file with SKU data from the provided DataFrame.

    Parameters
    ----------
    file_path : str
        Path to the CSV file to populate.
    df : pd.DataFrame
        DataFrame containing SKU data to write to the CSV.
    debug : bool, optional
        If True, prints detailed diagnostic information. Default is False.

    Returns
    -------
    str or None
        Returns the file path if the CSV was populated successfully, or None if there was an error or the file is empty.

    Notes
    -----
    - Appends the DataFrame to the specified CSV file.
    - If the file does not exist, writes the header; otherwise, appends without header.
    - Verifies that the file was populated by reading it back and checking for rows.
    - Prints warnings or errors if the file is missing or empty, or if verification fails.
    """
    
    if debug:
        print(f"Populating CSV at: {file_path} with DataFrame of shape: {df.shape}")
    
    # Write the DataFrame to the CSV, appending if file exists
    df.to_csv(file_path, mode='a', index=False, header=not os.path.exists(file_path))
    
    if debug:
        print(f"Successfully populated CSV at: {file_path}")    

    # Verify if file was populated successfully
    try:
        # Check if file exists and has content
        if os.path.exists(file_path):
            # Read the file to verify it has data
            verification_df = pd.read_csv(file_path)
            
            if len(verification_df) > 0:
                if debug:
                    print(f"Verification: File contains {len(verification_df)} rows")
                return file_path
            else:
                print(f"WARNING: File exists but is empty: {file_path}")
                return None
        else:
            print(f"WARNING: File does not exist: {file_path}")
            return None
    except Exception as e:
        print(f"ERROR: Failed to verify file: {e}")
        return None

# Test the function
test_batch_name = batch_name_list[0]
test_batch_df = batch_to_dataframe(test_batch_name, batch_dict)
populate_output_csv(file_path, test_batch_df, debug=True)
    

No duplicate SKUs found in batch 'Legendary Distressed Hat'. Total SKUs: 2

Populating CSV at: Results/Bandit Truck Logo Tee_skus_20251201.csv with DataFrame of shape: (2, 8)
Successfully populated CSV at: Results/Bandit Truck Logo Tee_skus_20251201.csv
Verification: File contains 2 rows


'Results/Bandit Truck Logo Tee_skus_20251201.csv'

Factor above functions as helpers into a single function

In [19]:
def generate_output_csv(output_dir='', batch_name='', df=pd.DataFrame(), overwrite=False, debug=False):
    """
    Generate and populate an output CSV file for a given batch DataFrame.
    
    Parameters
    ----------
    output_dir : str
        Directory path where the CSV should be created (e.g., 'Results/').
    batch_name : str
        Name of the product model/batch being processed.
    df : pd.DataFrame
        DataFrame containing SKU data to write to the CSV.
    overwrite : bool, optional
        If True, overwrites existing file; if False, skips if file exists. Default is False.
    debug : bool, optional
        If True, prints detailed diagnostic information. Default is False.
    
    Returns
    -------
    str or None
        Absolute path to the created and populated CSV file if successful, or None on failure.
    """
    
    # Initialize the output CSV
    file_path = initialize_output_csv(output_dir, batch_name, overwrite, debug)
    
    if file_path is None:
        if debug:
            print(f"Skipping population: File already exists and overwrite=False")
        return None
    
    # Populate the CSV with the provided DataFrame
    populated_path = populate_output_csv(file_path, df, debug)
    
    if populated_path is not None:
        if debug:
            print(f"Successfully generated and populated CSV at: {populated_path}")
        return populated_path
    else:
        if debug:
            print(f"Failed to populate CSV at: {file_path}")
        return None
    
# Test the function
test_batch_name = batch_name_list[0]
test_batch_df = batch_to_dataframe(test_batch_name, batch_dict)
generate_output_csv('Results/', test_batch_name, test_batch_df, overwrite=True, debug=True)

No duplicate SKUs found in batch 'Legendary Distressed Hat'. Total SKUs: 2

Debug: Attempting to create file at: Results/Legendary Distressed Hat_skus_20251201.csv
Debug: Directory exists: True
Debug: File exists: False
Debug: Overwrite mode: True
Created new CSV template with columns: ['Index', 'SKU', 'Main_Category', 'Sub_Category', 'Name', 'Size', 'Fit', 'Color', 'Design', 'Material', 'Scent']
Saved to: Results/Legendary Distressed Hat_skus_20251201.csv
Populating CSV at: Results/Legendary Distressed Hat_skus_20251201.csv with DataFrame of shape: (2, 8)
Successfully populated CSV at: Results/Legendary Distressed Hat_skus_20251201.csv
Verification: File contains 2 rows
Successfully generated and populated CSV at: Results/Legendary Distressed Hat_skus_20251201.csv


'Results/Legendary Distressed Hat_skus_20251201.csv'

## SKU number capture test

run the above functions to generate SKUs and write them into the initialized csv file.

### Scripting the Functions

In [20]:
# Load the data
catalog_data_source = "CHARTS/catalog.numbers"
tables = load_catalog_data(catalog_data_source)

# Validate and extract feature lists
code_sheet_list, catalog_feature_list = validate_catalog_features(tables)

# Prepare batch names
product_models_data_source = tables['Product_Catalog']
batch_name_list = prepare_batches(product_models_data_source) 

# Generate batch dictionary
batch_dict = generate_batch_dict(batch_name_list)

# Parse batches into dataframes
batch_df_list = []
for batch in batch_dict:
    batch_df = batch_to_dataframe(batch,batch_dict)
    batch_df_list.append(batch_df)

# output batches to CSV files
for df in batch_df_list:
    generate_output_csv(output_dir='Results/', batch_name=df.name.replace('_df',''), df=df, overwrite=True, debug=True)
    print(f"DataFrame Name: {df.name}, Shape: {df.shape}")




Sheet names and catalog features MATCH exactly!
All product names are unique, total batches to process: 18
No duplicate SKUs found in batch 'Legendary Distressed Hat'. Total SKUs: 2

No duplicate SKUs found in batch 'Bandit Banquet Tee'. Total SKUs: 12

No duplicate SKUs found in batch 'Bandit Truck Wreath Tee'. Total SKUs: 12

No duplicate SKUs found in batch 'Bandit Truck Jacket'. Total SKUs: 12

No duplicate SKUs found in batch 'Bandit Truck Flex Fit Hat'. Total SKUs: 2

No duplicate SKUs found in batch 'Legendary Trucker Hat'. Total SKUs: 2

No duplicate SKUs found in batch 'Collectible Gold Keychain'. Total SKUs: 2

No duplicate SKUs found in batch 'Legendary Long Sleeve Tee'. Total SKUs: 12

No duplicate SKUs found in batch 'Legendary Jacket'. Total SKUs: 12

No duplicate SKUs found in batch 'Legendary Tee'. Total SKUs: 48

No duplicate SKUs found in batch 'Bandit Truck Logo Tee'. Total SKUs: 12

No duplicate SKUs found in batch 'Legendary Sweater'. Total SKUs: 12

No duplicate S

In [15]:

for df in batch_df_list:
    initialize_output_csv(output_dir='Results/', batch_name=df.name.replace('_df',''), overwrite=True, debug=True)
    print(f"DataFrame Name: {df.name}, Shape: {df.shape}")


Debug: Attempting to create file at: Results/Legendary Long Sleeve Tee_skus_20251121.csv
Debug: Directory exists: True
Debug: File exists: True
Debug: Overwrite mode: True
Overwriting existing CSV at: Results/Legendary Long Sleeve Tee_skus_20251121.csv
Created new CSV template with columns: ['Index', 'SKU', 'Main_Category', 'Sub_Category', 'Name', 'Size', 'Fit', 'Color', 'Design', 'Material', 'Scent']
Saved to: Results/Legendary Long Sleeve Tee_skus_20251121.csv
DataFrame Name: Legendary Long Sleeve Tee_df, Shape: (8, 9)
Debug: Attempting to create file at: Results/Bandit Truck Logo Tee_skus_20251121.csv
Debug: Directory exists: True
Debug: File exists: True
Debug: Overwrite mode: True
Overwriting existing CSV at: Results/Bandit Truck Logo Tee_skus_20251121.csv
Created new CSV template with columns: ['Index', 'SKU', 'Main_Category', 'Sub_Category', 'Name', 'Size', 'Fit', 'Color', 'Design', 'Material', 'Scent']
Saved to: Results/Bandit Truck Logo Tee_skus_20251121.csv
DataFrame Name: Ba

### Write generated SKUS to CSV

Start with a Pandas dataframe